# *Deliverable #2: Create a customer travel destinations map*

## Setup & Dependencies

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import json
import matplotlib as plt

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# Check that api_key is correct
#g_key

## NOTE: Be sure to comment out this command & to delete output from file before pushing to GitHub

In [3]:
# Check gmaps installation
gmaps.__version__

'0.9.0'

## Import Source Data

In [4]:
# Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Ushuaia,-54.8000,-68.3000,40.66,75,40,11.50,AR,2022-07-28 05:17:43
1,1,Hilo,19.7297,-155.0900,79.29,82,40,3.44,US,2022-07-28 05:17:35
2,2,Punta Arenas,-53.1500,-70.9167,39.31,81,20,13.80,CL,2022-07-28 05:20:53
3,3,Faya,18.3851,42.4509,68.52,47,100,7.02,SA,2022-07-28 05:20:53
4,4,Port Hardy,50.6996,-127.4199,60.98,88,20,0.00,CA,2022-07-28 05:20:54


## Filter Data for Weather Preferences (Input Statements)

In [5]:
# Add new columns to DF to hold user inputs
city_data_df['min_temp'] = ""
city_data_df['max_temp'] = ""

city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,min_temp,max_temp
0,0,Ushuaia,-54.8000,-68.3000,40.66,75,40,11.50,AR,2022-07-28 05:17:43,,
1,1,Hilo,19.7297,-155.0900,79.29,82,40,3.44,US,2022-07-28 05:17:35,,
2,2,Punta Arenas,-53.1500,-70.9167,39.31,81,20,13.80,CL,2022-07-28 05:20:53,,
3,3,Faya,18.3851,42.4509,68.52,47,100,7.02,SA,2022-07-28 05:20:53,,
4,4,Port Hardy,50.6996,-127.4199,60.98,88,20,0.00,CA,2022-07-28 05:20:54,,


In [6]:
# Prompt the user to enter minimum acceptable temperature 
#min_temp = float(input("What is the coolest acceptable temperature for your vacation?"))

## NOTE: There is no "Min Temp" data in the city_data_df DF...

In [7]:
# Prompt the user to enter maximum acceptable temperature 
max_temp = float(input("What is the hottest acceptable temperature for your vacation?"))

What is the hottest acceptable temperature for your vacation?95


In [8]:
# Filter the city_data_df DataFrame using the input statements to create a new DataFrame.
#filtered_city_data_df = city_data_df.loc[(city_data_df[min_temp] >= min_temp) &
#                                        (city_data_df[max_temp] <= max_temp)]

filtered_city_data_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp)]
filtered_city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,min_temp,max_temp
0,0,Ushuaia,-54.8000,-68.3000,40.66,75,40,11.50,AR,2022-07-28 05:17:43,,
1,1,Hilo,19.7297,-155.0900,79.29,82,40,3.44,US,2022-07-28 05:17:35,,
2,2,Punta Arenas,-53.1500,-70.9167,39.31,81,20,13.80,CL,2022-07-28 05:20:53,,
3,3,Faya,18.3851,42.4509,68.52,47,100,7.02,SA,2022-07-28 05:20:53,,
4,4,Port Hardy,50.6996,-127.4199,60.98,88,20,0.00,CA,2022-07-28 05:20:54,,


## Clean Filtered Data

In [9]:
# Determine if there are any empty rows.
filtered_city_data_df.count()

City_ID       683
City          683
Lat           683
Lng           683
Max Temp      683
Humidity      683
Cloudiness    683
Wind Speed    683
Country       679
Date          683
min_temp      683
max_temp      683
dtype: int64

In [10]:
# Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
filtered_city_data_df = filtered_city_data_df.dropna()
filtered_city_data_df.count()

City_ID       679
City          679
Lat           679
Lng           679
Max Temp      679
Humidity      679
Cloudiness    679
Wind Speed    679
Country       679
Date          679
min_temp      679
max_temp      679
dtype: int64

## Retrieve hotel data (Python *Requests* & *JSON Traversals*) 

In [11]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = filtered_city_data_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Ushuaia,AR,40.66,-54.8000,-68.3000,
1,Hilo,US,79.29,19.7297,-155.0900,
2,Punta Arenas,CL,39.31,-53.1500,-70.9167,
3,Faya,SA,68.52,18.3851,42.4509,
4,Port Hardy,CA,60.98,50.6996,-127.4199,
5,Chuy,UY,61.45,-33.6971,-53.4616,
6,Busselton,AU,65.01,-33.6500,115.3333,
7,Herat,AF,87.82,34.5000,62.0000,
8,Coihaique,CL,39.15,-45.5752,-72.0662,
9,Naze,JP,86.92,28.3667,129.4833,


In [12]:
# Set parameters (Dict Keys) to search for hotels within 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
#    print(params)
#    print(index)
    
    # Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except IndexError:
        print("Hotel Not found... skipping...")   

Hotel Not found... skipping...
Hotel Not found... skipping...
Hotel Not found... skipping...
Hotel Not found... skipping...
Hotel Not found... skipping...
Hotel Not found... skipping...
Hotel Not found... skipping...
Hotel Not found... skipping...
Hotel Not found... skipping...
Hotel Not found... skipping...
Hotel Not found... skipping...
Hotel Not found... skipping...
Hotel Not found... skipping...
Hotel Not found... skipping...
Hotel Not found... skipping...
Hotel Not found... skipping...
Hotel Not found... skipping...
Hotel Not found... skipping...
Hotel Not found... skipping...
Hotel Not found... skipping...
Hotel Not found... skipping...
Hotel Not found... skipping...
Hotel Not found... skipping...
Hotel Not found... skipping...
Hotel Not found... skipping...
Hotel Not found... skipping...
Hotel Not found... skipping...
Hotel Not found... skipping...
Hotel Not found... skipping...
Hotel Not found... skipping...
Hotel Not found... skipping...
Hotel Not found... skipping...
Hotel No

In [13]:
# View results of JSON query
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Ushuaia,AR,40.66,-54.8000,-68.3000,Albatross Hotel
1,Hilo,US,79.29,19.7297,-155.0900,Hilo Hawaiian Hotel
2,Punta Arenas,CL,39.31,-53.1500,-70.9167,Hotel Hain
3,Faya,SA,68.52,18.3851,42.4509,املاك خالد الشعشوع
4,Port Hardy,CA,60.98,50.6996,-127.4199,Airport Inn-Port Hardy
5,Chuy,UY,61.45,-33.6971,-53.4616,Nuevo Hotel Plaza
6,Busselton,AU,65.01,-33.6500,115.3333,Observatory Guest House
7,Herat,AF,87.82,34.5000,62.0000,
8,Coihaique,CL,39.15,-45.5752,-72.0662,Diego De Almagro Coyhaique
9,Naze,JP,86.92,28.3667,129.4833,Shishime Hotel


## Clean-up retrieved hotel data

In [14]:
# Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
hotel_df = hotel_df.dropna()
hotel_df.count()

City          679
Country       679
Max Temp      679
Lat           679
Lng           679
Hotel Name    679
dtype: int64

In [15]:
# Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# Export the City_Data into a csv
hotel_df.to_csv(output_data_file)

## Retrieve Data for WeatherPy_vacation (marker layer) map

In [16]:
# Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Hotel Name</dt><dd>{Hotel Name} at {Max Temp}</dd>
"""

# Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

## Create the WeatherPy_vacation (marker layer) map

In [17]:
# Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)

# Define fig object
fig = gmaps.figure()

# Add the layer to the map
fig.add_layer(marker_layer)

# Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [20]:
#Import matplotlib
import matplotlib as plt

In [22]:
# Save the figure to a .png file
# plt.savefig("Vacation_Search/WeatherPy_vacation_map.png")